<a href="https://colab.research.google.com/github/LaurentTits/AI_execices/blob/main/DB3_NASNetMobile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
!rm -rf sample_data

Wed Nov  2 15:59:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2 
import csv
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import math
from PIL import Image
from google.colab import files, drive
print("Tensorflow version: "+tf.__version__)
print("Keras version: " + tf.keras.__version__)

drive.mount('/content/gdrive')

bases_path_after="bases"
if os.path.exists(bases_path_after) == False:
    os.makedirs(bases_path_after)

!rm -rf sample_data
! wget --no-check-certificate https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
! tar xf FIRE_DATABASE_3.tar -C 'bases' --one-top-level
! rm FIRE_DATABASE_3.tar

!rm -rf sample_data
!wget --no-check-certificate http://195.154.53.219/downloads/test.tar
! tar xf test.tar -C 'bases' --one-top-level
! rm test.tar

# !printf '%s\n' 'fire' 'no_fire' 'start_fire'> classes.txt
!printf '%s\n' 'fire' 'no_fire' 'start_fire'> /content/gdrive/MyDrive/classes.txt

Tensorflow version: 2.9.2
Keras version: 2.9.0
Mounted at /content/gdrive
--2022-11-02 16:01:01--  https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
Resolving cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)... 193.190.209.220
Connecting to cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471111680 (449M) [application/octet-stream]
Saving to: ‘FIRE_DATABASE_3.tar’

FIRE_DATABASE_3.tar 100%[===================>] 449.29M  20.1MB/s    in 37s     

2022-11-02 16:01:39 (12.2 MB/s) - ‘FIRE_DATABASE_3.tar’ saved [471111680/471111680]

--2022-11-02 16:01:41--  http://195.154.53.219/downloads/test.tar
Connecting to 195.154.53.219:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58196480 (56M) [application/octet-stream]
Saving to: ‘test.tar’

test.tar            100%[===================>]  55.50M  15.4MB/s    in 3.6s    

2022-11-02 16:01:45 (15.4 MB/s) - ‘test.tar’ 

In [35]:
nb_classes = 3
batch_size=8 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
epochs=5 #@param [5, 10,20,50,100,200] {type:"raw"}
dataset_path = "bases"
input_dim=224 #@param [224,299] {type:"raw"}  
dataset_name='FIRE_DATABASE_3' #@param ["small","FIRE_DATABASE_3","DB2","DB3"]
train_dataset = os.path.join('bases/', dataset_name)
test_name='test' #@param ["test","test1","test2","test3"]
test_dataset = os.path.join('bases/', test_name)
classes_path = "/content/gdrive/MyDrive/classes.txt"
classifier = "NASNetLarge" #@param ["Xception","VGG16","VGG19","ResNet50","ResNet121","ResNet152","ResNet50V2","ResNet121V2","ResNet152V2","InceptionV3","InceptionResNetV2","MobileNet","MobileNetV2","DenseNet121","DenseNet169","DenseNet201","NASNetLarge","NASNetMobile"] {type:"string"}


In [11]:
# Récupérer les noms des classes
with open(classes_path, 'r') as f:
    classes = f.readlines()
    classes = list(map(lambda x: x.strip(), classes))
num_classes = len(classes)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
	train_dataset,               # chemin vers le jeu de données
	validation_split=0.2,        # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="training",           # Sélection du jeu de données d'entraînement
	seed=42,                     # Initialisation du générateur aléatoire (permutations)
	image_size=(input_dim, input_dim),   # Taille des images d'entrée
	batch_size=batch_size,   # Taille du mini-batch
  label_mode='categorical'    # Conversion au format One-Hot
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
	train_dataset,               # chemin vers le jeu de données
	validation_split=0.2,        # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="validation",           # Sélection du jeu de données d'entraînement
	seed=42,                     # Initialisation du générateur aléatoire (permutations)
	image_size=(input_dim, input_dim),   # Taille des images d'entrée
	batch_size=batch_size,   # Taille du mini-batch
  label_mode='categorical'    # Conversion au format One-Hot
)


Found 1500 files belonging to 3 classes.
Using 1200 files for training.
Found 1500 files belonging to 3 classes.
Using 300 files for validation.


In [36]:
from tensorflow.keras.applications.nasnet import NASNetMobile, preprocess_input, decode_predictions

model_imagenet = NASNetMobile(weights='imagenet', include_top=False, input_shape = (input_dim, input_dim, 3))   # 24,226,656 bytes

x = model_imagenet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(1056, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)
model = Model(inputs=model_imagenet.inputs, outputs=predictions)


#freeze the model except two last layers
for layer in model.layers[:-2]:
  layer.trainable = False



In [37]:
print(model.summary())

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 111, 111, 32  864         ['input_16[0][0]']               
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 111, 111, 32  128         ['stem_conv1[0][0]']             
                                )                                                          

In [38]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [34]:
stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, 
                        verbose=1, mode='auto')
#ckpt_save = os.path.join(".",'model_fine_ep{epoch}_valloss{val_loss:.3f}.h5')
ckpt_save = os.path.join("/content/gdrive/MyDrive",'NASNetMobile_db3_ep{epoch}_valloss{val_loss:.3f}.h5')

checkpoint = ModelCheckpoint(ckpt_save, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

callback_list = [checkpoint, stopper]

history = model.fit(train_ds, validation_data = val_ds, batch_size = batch_size, 
                    epochs=epochs, callbacks=callback_list)

Epoch 1/5


ValueError: ignored